# Quick Start

Below is a sample demo of interaction with the environment.

In [1]:
from maro.simulator import Env
from maro.simulator.scenarios.cim.common import Action, DecisionEvent

env = Env(scenario="cim", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

metrics: object = None
decision_event: DecisionEvent = None
is_done: bool = False

while not is_done:
    action: Action = None
    metrics, decision_event, is_done = env.step(action)

print(metrics)

{'order_requirements': 200000, 'container_shortage': 100000, 'operation_number': 0}


# Environment of Container Inventory Management (CIM)

To initialize an environment, you need to specify the values of several parameters:
- **scenario**: The target scenario of this Env.
  - `cim` denotes for the Container Inventory Management (CIM).
- **topology**: The target topology of this Env. As shown below, you can get the predefined topology list by calling `get_topologies(scenario='cim')`.
- **start_tick**: The start tick of this Env, 1 tick corresponds to 1 day in cim.
  - In the demo above, `start_tick=0` indicates a simulation start from the beginning of the given topology.
- **durations**: The duration of this Env, in the unit of tick/day.
  - In the demo above, `durations=100` indicates a simulation length of 100 days.

You can get all available scenarios and topologies by calling:

In [2]:
from maro.simulator.utils import get_scenarios, get_topologies
from pprint import pprint
from typing import List

scenarios: List[str] = get_scenarios()
topologies: List[str] = get_topologies(scenario='cim')

pprint(f'The available scenarios in MARO:')
pprint(scenarios)

print()
pprint(f'The predefined topologies in CIM:')
pprint(topologies)

'The available scenarios in MARO:'
['cim', 'citi_bike']

'The predefined topologies in CIM:'
['global_trade.22p_l0.0',
 'global_trade.22p_l0.1',
 'global_trade.22p_l0.2',
 'global_trade.22p_l0.3',
 'global_trade.22p_l0.4',
 'global_trade.22p_l0.5',
 'global_trade.22p_l0.6',
 'global_trade.22p_l0.7',
 'global_trade.22p_l0.8',
 'toy.4p_ssdd_l0.0',
 'toy.4p_ssdd_l0.1',
 'toy.4p_ssdd_l0.2',
 'toy.4p_ssdd_l0.3',
 'toy.4p_ssdd_l0.4',
 'toy.4p_ssdd_l0.5',
 'toy.4p_ssdd_l0.6',
 'toy.4p_ssdd_l0.7',
 'toy.4p_ssdd_l0.8',
 'toy.5p_ssddd_l0.0',
 'toy.5p_ssddd_l0.1',
 'toy.5p_ssddd_l0.2',
 'toy.5p_ssddd_l0.3',
 'toy.5p_ssddd_l0.4',
 'toy.5p_ssddd_l0.5',
 'toy.5p_ssddd_l0.6',
 'toy.5p_ssddd_l0.7',
 'toy.5p_ssddd_l0.8',
 'toy.6p_sssbdd_l0.0',
 'toy.6p_sssbdd_l0.1',
 'toy.6p_sssbdd_l0.2',
 'toy.6p_sssbdd_l0.3',
 'toy.6p_sssbdd_l0.4',
 'toy.6p_sssbdd_l0.5',
 'toy.6p_sssbdd_l0.6',
 'toy.6p_sssbdd_l0.7',
 'toy.6p_sssbdd_l0.8']


Once you created an instance of the environment, you can easily access the real-time information of this environment, like:

In [3]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for CIM scenario
env = Env(scenario="cim", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

# The current tick
tick: int = env.tick
print(f"The current tick: {tick}.")

# The current frame index, which indicates the index of current frame in the snapshot-list
frame_index: int = env.frame_index
print(f"The current frame index: {frame_index}.")

# The agent index list in the environment
agent_idx_list: List[int] = env.agent_idx_list
print(f"There are {len(agent_idx_list)} agents in this Env.")

# The whole snapshot-list of the environment, snapshots are taken in the granularity of the given snapshot_resolution
# The example of how to use the snapshot will be shown later
snapshot_list: SnapshotList = env.snapshot_list
print(f"There will be {len(snapshot_list)} snapshots in total.")

# The summary info of the environment
summary: dict = env.summary
print(f"\nEnv Summary:")
pprint(summary)

# The metrics of the environment
metrics: dict = env.metrics
print(f"\nEnv Metrics:")
pprint(metrics)

The current tick: 0.
The current frame index: 0.
There are 5 agents in this Env.
There will be 100 snapshots in total.

Env Summary:
{'node_detail': {'matrices': {'attributes': {'full_on_ports': {'slots': 25,
                                                               'type': 'i'},
                                             'full_on_vessels': {'slots': 30,
                                                                 'type': 'i'},
                                             'vessel_plans': {'slots': 30,
                                                              'type': 'i'}},
                              'number': 1},
                 'ports': {'attributes': {'acc_booking': {'slots': 1,
                                                          'type': 'i'},
                                          'acc_fulfillment': {'slots': 1,
                                                              'type': 'i'},
                                          'acc_shortage': {'slots': 1

# Interaction with the environment

Before starting interaction with the environment, we need to know `DecisionEvent` and `Action` first.

## DecisionEvent

Once the environment need the agent's response to promote the simulation, it will throw an `DecisionEvent`. In the scenario of CIM, the information of each `DecisionEvent` is listed as below:
- **tick**: (int) The corresponding tick;
- **port_idx**: (int) The id of the port/agent that needs to respond to the environment;
- **vessel_idx**: (int) The id of the vessel/operation object of the port/agnet;
- **action_scope**: (ActionScope) ActionScope has two attributes:
  - `load` indicates the maximum quantity that can be loaded from the port the vessel;
  - `discharge` indicates the maximum quantity that can be discharged from the vessel to the port;
- **early_discharge**: (int) When the available capacity in the vessel is not enough to load the ladens, some of the empty containers in the vessel will be early discharged to free the space. The quantity of empty containers that have been early discharged due to the laden loading is recorded in this field.

## Action

Once we get a `DecisionEvent` from the environment, we should respond with an `Action`. Valid `Action` could be:

- `None`, which means do nothing.
- A valid `Action` instance, including:
   - **vessel_idx**: (int) The id of the vessel/operation object of the port/agent;
   - **port_idx**: (int) The id of the port/agent that take this action;
   - **quantity**: (int) The sign of this value denotes different meanings:
      - Positive quantity means unloading empty containers from vessel to port;
      - Negative quantity means loading empty containers from port to vessel.

## Generate random actions based on the DecisionEvent

The demo code in the Quick Start part has shown an interaction mode that doing nothing(responding with `None` action). Here we read the detailed information from the `DecisionEvent` and generate random `Action` based on it.

In [4]:
from maro.simulator import Env
from maro.simulator.scenarios.cim.common import Action, DecisionEvent

import random

# Initialize an Env for cim scenario
env = Env(scenario="cim", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

metrics: object = None
decision_event: DecisionEvent = None
is_done: bool = False
action: Action = None

# Start the env with a None Action
metrics, decision_event, is_done = env.step(None)

while not is_done:
    # Generate a random Action according to the action_scope in DecisionEvent
    random_quantity = random.randint(
        -decision_event.action_scope.load,
        decision_event.action_scope.discharge
    )
    action = Action(
        vessel_idx=decision_event.vessel_idx,
        port_idx=decision_event.port_idx,
        quantity=random_quantity
    )
    
    # Randomly sample some records to show in the output
    if random.random() > 0.95:
        print(f"*************\n{decision_event}\n{action}")
    
    # Respond the environment with the generated Action
    metrics, decision_event, is_done = env.step(action)

*************
DecisionEvent(tick=14, port_idx=0, vessel_idx=5, action_scope=ActionScope(load=8856, discharge=1981))
Action(port_idx=0, vessel_idx=5, quantity=-2667)
*************
DecisionEvent(tick=21, port_idx=2, vessel_idx=1, action_scope=ActionScope(load=15997, discharge=3061))
Action(port_idx=2, vessel_idx=1, quantity=-11608)


## Get the environment observation

You can also implement other strategies or build models to take action. At this time, real-time information and historical records of the environment are very important for making good decisions. In this case, the the environment snapshot list is exactly what you need.

The information in the snapshot list is indexed by 3 dimensions:
- A tick index (list). (int / List[int]) Empty indicates for all time slides till now;
- A port id (list). (int / List[int]) Empty indicates for all ports/agents;
- An attribute name (list). (str / List[str]) You can get all available attributes in `env.summary` as shown before.

The return value from the snapshot list is a numpy.ndarray with shape **(num_tick * num_port * num_attribute, )**.

More detailed introduction to the snapshot list is [here](https://maro.readthedocs.io/en/latest/key_components/data_model.html#advanced-features).

In [5]:
from maro.simulator import Env
from pprint import pprint

env = Env(scenario="cim", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

# To get the attribute list that can be accessed in snapshot_list
pprint(env.summary['node_detail'], depth=2)
print()
# The attribute list of ports
pprint(env.summary['node_detail']['ports'])

{'matrices': {'attributes': {...}, 'number': 1},
 'ports': {'attributes': {...}, 'number': 5},
 'vessels': {'attributes': {...}, 'number': 6}}

{'attributes': {'acc_booking': {'slots': 1, 'type': 'i'},
                'acc_fulfillment': {'slots': 1, 'type': 'i'},
                'acc_shortage': {'slots': 1, 'type': 'i'},
                'booking': {'slots': 1, 'type': 'i'},
                'capacity': {'slots': 1, 'type': 'i'},
                'empty': {'slots': 1, 'type': 'i'},
                'fulfillment': {'slots': 1, 'type': 'i'},
                'full': {'slots': 1, 'type': 'i'},
                'on_consignee': {'slots': 1, 'type': 'i'},
                'on_shipper': {'slots': 1, 'type': 'i'},
                'shortage': {'slots': 1, 'type': 'i'},
                'transfer_cost': {'slots': 1, 'type': 'f'}},
 'number': 5}


In [6]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for CIM scenario
env = Env(scenario="cim", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

# Start the environment with None action
_, decision_event, is_done = env.step(None)

while not is_done:
    # Case of access snapshot after a certain number of ticks
    if env.tick >= 80:
        # The tick list of past 1 week
        past_week_ticks = [x for x in range(env.tick - 7, env.tick)]
        # The port index of the current decision_event
        decision_port_idx = decision_event.port_idx
        # The attribute list to access 
        intr_port_infos = ["booking", "empty", "shortage"]

        # Query the snapshot list of this environment to get the information of
        # the booking, empty, shortage of the decision port in the past week
        past_week_info = env.snapshot_list["ports"][
            past_week_ticks : decision_port_idx : intr_port_infos
        ]
        pprint(past_week_info)
        
        # This demo code is used to show how to access the information in snapshot,
        # so we terminate the env here for clear output
        break

    # Drive the environment with None action
    _, decision_event, is_done = env.step(None)

array([1000.,    0., 1000., 1000.,    0., 1000., 1000.,    0., 1000.,
       1000.,    0., 1000., 1000.,    0., 1000., 1000.,    0., 1000.,
       1000.,    0., 1000.], dtype=float32)
